Load graphml file and transform into h5 file using pandana

In [13]:
import networkx as nx

G_temp=nx.read_graphml("gipuzkoa_walk_with_geometry.graphml")

In [14]:
import pandas as pd

node_df=pd.DataFrame.from_dict(dict(G_temp.nodes(data=True)), orient="index")
edge_df=nx.to_pandas_edgelist(G_temp)

In [15]:
print(node_df.head())

                 y           x street_count highway  ref
448001  43.1795625  -2.4899828            3     NaN  NaN
448006  43.1826552  -2.4814224            4     NaN  NaN
451277  43.2049133   -2.425101            3     NaN  NaN
451289  43.1927813  -2.4381722            3     NaN  NaN
469501  43.1911116  -2.4516244            3     NaN  NaN


In [16]:
# rename length to distance
edge_df=edge_df.rename(columns={"length": "distance"})
print(edge_df.head())

   source      target reversed bridge  \
0  448001  3934729820    False    NaN   
1  448001    25438648     True    yes   
2  448001      916343     True    NaN   
3  448006  5949227426     True    NaN   
4  448006     6928916     True    NaN   

                                            geometry      highway junction  \
0  LINESTRING (-2.4899828 43.1795625, -2.489874 4...     tertiary      NaN   
1  LINESTRING (-2.4899828 43.1795625, -2.4900901 ...  residential      NaN   
2  LINESTRING (-2.4899828 43.1795625, -2.4901355 ...     tertiary      NaN   
3  LINESTRING (-2.4814224 43.1826552, -2.4816299 ...     tertiary      NaN   
4  LINESTRING (-2.4814224 43.1826552, -2.4814342 ...  residential      NaN   

  access maxspeed                             osmid      ref tunnel width  \
0    NaN      NaN                          44716332      NaN    NaN   NaN   
1    NaN      NaN  [219375572, 219375573, 44716327]      NaN    NaN   NaN   
2    NaN      NaN                        1009087270  

In [17]:
# Convert columns to numeric, forcing errors='coerce' to handle non-numeric values
edge_df["distance"] = pd.to_numeric(edge_df["distance"], errors="coerce")

In [20]:
# just keep index, x and y from node_df
node_df=node_df[["x", "y"]]

# just keep source, target and distance from edge_df
edge_df=edge_df[["source", "target", "distance"]]

In [21]:
print(node_df.head())
print(edge_df.head())

                 x           y
448001  -2.4899828  43.1795625
448006  -2.4814224  43.1826552
451277   -2.425101  43.2049133
451289  -2.4381722  43.1927813
469501  -2.4516244  43.1911116
   source      target  distance
0  448001  3934729820   113.604
1  448001    25438648   139.660
2  448001      916343    42.152
3  448006  5949227426    20.781
4  448006     6928916    78.047


In [22]:
import pandana as pdna

node_x=node_df["x"]
node_y=node_df["y"]
edge_from=edge_df["source"]
edge_to=edge_df["target"]
edge_weights=pd.DataFrame(edge_df["distance"])

walk_network = pdna.Network(node_x, node_y, edge_from, edge_to, edge_weights, twoway=True)

print(walk_network)

In [23]:
network_file="walk_network.h5"
walk_network.save_hdf5(network_file)